In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe

import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import java.util.UUID.randomUUID

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val ACCOUNT_MATCHERS_CSV_FILE = "../../data/provision/account-matchers.csv"
val FUND_MATCHERS_CSV_FILE = "../../data/provision/fund-matchers.csv"

// TODO(Johann) create a user
val userId = randomUUID()

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

In [2]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(userId).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(userId, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=191f4eed-da77-4780-874c-e54456ce2f08, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=99ebc53e-2708-4044-94f1-e5eaf8cffa25, name=ING RON, unit=Currency(value=RON))
AccountTO(id=3e939065-5486-443f-8d8f-aef08a6d1dfe, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=c6db0730-cd27-44cd-9090-1a13fdcb8d29, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=f2e66d94-afc4-4bd2-8212-0accf20a6be4, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=086d0947-a27f-4308-8d41-5a5a8379dc86, name=Other, unit=Currency(value=RON))

In [3]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    // TODO(Johann) fix npe on get existing funds without accounts
    val existingFunds = fundSdk.listFunds(userId).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(userId, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=3d42f202-52a9-4d78-96ff-a075be0675a2, name=Expenses)
FundTO(id=770a1de0-428d-4d21-ba5c-b8b93d562dc5, name=Work Income)
FundTO(id=3557554d-b9fd-466a-a1e9-4eae76a2c84b, name=Savings)

In [4]:
import ro.jf.funds.importer.api.model.AccountMatcherTO

val accountMatchers = runBlocking {
    DataFrame.readCSV(ACCOUNT_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row -> AccountMatcherTO(row.getValue<String>("import_account_name"), AccountName(row.getValue<String>("account_name"))) }
}
accountMatchers.map { it.toString() }.joinToString(separator = "\n")

AccountMatcherTO(importAccountName=ING old, accountName=ING RON)
AccountMatcherTO(importAccountName=ING Economy old, accountName=ING Economy)
AccountMatcherTO(importAccountName=Cash RON, accountName=Cash RON)
AccountMatcherTO(importAccountName=Food Coupons, accountName=Food Coupons)
AccountMatcherTO(importAccountName=Revolut - Manual, accountName=Revolut RON)
AccountMatcherTO(importAccountName=Other, accountName=Other)

In [5]:
import ro.jf.funds.importer.api.model.*

val fundMatchers = runBlocking {
    DataFrame.readCSV(FUND_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_account" -> FundMatcherTO.ByAccount(
                    importAccountName = row.getValue<String>("import_account_name"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )
                
                "by_label" -> FundMatcherTO.ByLabel(
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label" -> FundMatcherTO.ByAccountLabel(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label_with_transfer" -> FundMatcherTO.ByAccountLabelWithTransfer(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    initialFundName = FundName(row.getValue<String>("initial_fund_name")),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                else -> error("fund matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByAccountLabelWithTransfer(importAccountName=ING Economy old, importLabel=Invest Profit Acc, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING old, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabel(importLabel=Gifts, fundName=Expenses)
ByAccount(importAccountName=ING old, fundName=Expenses)
ByAccount(importAccountName=ING Economy old, fundName=Expenses)
ByAccount(importAccountName=Cash RON, fundName=Expenses)
ByAccount(importAccountName=Other, fundName=Exp

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

// TODO(Johann) load all files from the directory
val csvFiles = listOf(
    File("../../data/wallet/2019/wallet_export_2019_01.csv"),
    File("../../data/wallet/2019/wallet_export_2019_02.csv"),
    File("../../data/wallet/2019/wallet_export_2019_03.csv"),
)
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    accountMatchers = accountMatchers,
    fundMatchers = fundMatchers
)
runBlocking {
    var importTask = importSdk.import(userId, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(userId, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=cdd29653-8d16-43f1-985c-71854fba3808, status=COMPLETED, reason=null)